# Metatranscriptomics Workflow

#### By Tomás Alonso Proschle Donoso (MSc. PUC Chile).

## 0. Environments installation using miniconda3

Miniconda3 is a lightweight distribution of the Conda package manager designed for Python and scientific computing. It allows you to create isolated environments with different versions of Python and packages. This is useful for managing dependencies and creating reproducible workflows. Miniconda3 simplifies dependency management, provides isolated environments for different programs, enables portability, and ensures version control. The version used was 23.3.1 and can be found on this [link](https://docs.conda.io/en/latest/miniconda.html).

For the preprocessing of the libraries, the following conda environment will be generated by running the following code on the console:

`
conda create --name preprocessing -c bioconda vsearch trimmomatic sortmerna bbmap fastqc multiqc
`

For the rna-seq procedure the following conda environment will be used:

`
conda create --name rna-seq -c bioconda hisat2 samtools htseq
`

## 1. Import readings

In [ ]:
mkdir -p 1_reads_raw

# The readings are imported to the 1_reads_raw folder, for this analysis the readings were imported locally

In [ ]:
# Create a list for the loop
ls 1_reads_raw/*.gz | sed 's/_..fastq.gz//; s/1_reads_raw.//' | sort -u > list

## 2. Pre-processing of readings

### 2.1 Analysis of readings using FastQC and MultiQC

Before starting to work with the readings, it is important to verify the quality of the samples. For this purpose, the FastQC program is used, which generates a complete report of important parameters such as the quality distribution of the reads. This report is individual for each file, so viewing one by one for a large number of files can be tedious, for this it is possible to use the MultiQC program which groups all the reports and delivers a summary of the results.

In [ ]:
# Activate the conda environment
conda activate preprocessing

In [ ]:
cd 1_reads_raw
mkdir -p fastqc_out

fastqc *.gz -out fastqc_out/
multiqc fastqc_out/ --outdir fastqc_out/

cd ..

The resulting file `multiqc_report.html` contains the display of the qualities of the readings, in this report it is possible to see the presence of adapters, average quality of the readings, presence of missmatches, among others. For RNA-Seq data it is very logical that there is a large number of duplicate reads, so the warning given can be ignored.

### 2.2 Trimming of poor quality readings using Trimmomatic

Looking at the previous report, it must be decided whether to cut or not, what minimum score to use and if it is necessary to remove adapters. In this case the Trimmomatic program is not instructed to remove Illumina adapters as it does not have them, this was possible to verify with the MultiQC report.

In [ ]:
mkdir -p 2_reads_trim

# Trimmomatic call
while read i; do
trimmomatic PE -threads 16 -trimlog 2_reads_trim/"$i".trimlog 1_reads_raw/"$i"_1.fastq.gz 1_reads_raw/"$i"_2.fastq.gz 2_reads_trim/"$i"_1_trim.fastq.gz 2_reads_trim/"$i"_1_trim_unpaired.fastq.gz 2_reads_trim/"$i"_2_trim.fastq.gz 2_reads_trim/"$i"_2_trim_unpaired.fastq.gz LEADING:3 TRAILING:3 SLIDINGWINDOW:3:15 MINLEN:36
done < list

# The evaluation of the readings is performed again
mkdir -p 2_reads_trim/fastqc_out
cd 2_reads_trim
ls *trim.fastq.gz | xargs fastqc --outdir fastqc_out/

# MultiQC report
multiqc fastqc_out/ --outdir fastqc_out/

cd ..

### 2.3 Filter readings using VSearch

After trimming the readings, a filter is performed taking into account the average expected error of the reading predicted by the phred score. For more information visit the following [page](https://www.drive5.com/usearch/manual/exp_errs.html).

In [ ]:
mkdir -p 3_reads_preproc

while read i; do
vsearch --fastq_filter 2_reads_trim/"$i"_1_trim.fastq.gz --reverse 2_reads_trim/"$i"_2_trim.fastq.gz --gzip_decompress --threads 0 --fastq_maxee 2.0 --fastqout 3_reads_preproc/"$i"_1.fastq --fastqout_rev 3_reads_preproc/"$i"_2.fastq
done < list

### 2.4 Separation of reads between rRNA and mRNA using SortMeRNA

Once the readings have been filtered for quality, it is necessary to separate those readings that correspond to rRNA, since the subsequent analysis is based on gene expression, which is closely linked to mRNA.

In [ ]:
# database download
mkdir -p 0_databases/sortmerna/

cd 0_databases/sortmerna/
wget https://github.com/biocore/sortmerna/releases/download/v4.3.4/database.tar.gz
tar -xzvf database.tar.gz

cd ../../

In [ ]:
cd 3_reads_preproc

while read i; do
mkdir -p "$i"
sortmerna --ref ../0_databases/sortmerna/smr_v4.3_sensitive_db.fasta --reads "$i"_1.fastq --reads "$i"_2.fastq --workdir "$i" --fastx --out2 --threads 16
done < ../list

cd ..

The files generated by the program contain reads that align with rRNA databases, so it is necessary to exclude them. For this purpose, the BBMap `filterbyname` script is used, which from the tags of the reads is able to create a new file with the information coming from mRNAs.

In [ ]:
cd 3_reads_preproc

while read i; do
filterbyname.sh in="$i"_1.fastq out="$i"_1_mRNA.fastq names="$i"/out/aligned_fwd.fq include=f
filterbyname.sh in="$i"_2.fastq out="$i"_2_mRNA.fastq names="$i"/out/aligned_rev.fq include=f
done < ../list

cd ..

Since it is possible that some of the readings may be missing due to some of the filters performed in the previous steps (Trimmomatic and VSearch), they are repaired using the `repair` script.

In [ ]:
mkdir -p 4_reads_prepared

while read i; do
repair.sh in=3_reads_preproc/"$i"_1_mRNA.fastq in2=3_reads_preproc/"$i"_2_mRNA.fastq out=4_reads_prepared/"$i"_1.fastq out2=4_reads_prepared/"$i"_2.fastq
done < list

### 2.5 Final quality check

For the last step of this section, the final quality of the libraries is verified before proceeding to the functional analysis.

In [ ]:
cd 4_reads_prepared
mkdir -p fastqc_out

fastqc *.fastq -out fastqc_out/
multiqc fastqc_out/

cd ..

## 3. Functional analysis

For the genes assignment the `rna-seq` environment will be used. The genomes of reference used are the following, which will be copied into a new genomes folder locally:

- [*Bifidobacterium bifidum* JCM 1254](https://www.bv-brc.org/view/Genome/398514.7)
- [*Bifidobacterium breve* DSM 20213 = JCM 1192](https://www.bv-brc.org/view/Genome/518634.20)
- [*Bifidobacterium longum* subsp. infantis ATCC 15697 = JCM 1222](https://www.bv-brc.org/view/Genome/391904.5)
- [*Bacteroides thetaiotaomicron* VPI-5482](https://www.bv-brc.org/view/Genome/226186.12)
- [*Lachnoclostridium symbiosum* WAL-14673](https://www.bv-brc.org/view/Genome/742741.3)
- [*Escherichia coli* str. K-12 substr. MG1655](https://www.bv-brc.org/view/Genome/511145.12)
- [*Pediococcus acidilactici* strain PMC65](https://www.bv-brc.org/view/Genome/1254.353)

The files downloaded for each genome were:

- Protein Sequences in FASTA (`.faa`)
- Genomic Sequences in FASTA (`.fna`)
- Genomic features in Generic Feature Format format (`.gff`)
- Pathway assignments in tab-delimited format (`.pathway.tab`)

In [ ]:
# environment activation
conda activate rna-seq

mkdir -p 5_functional_analysis

mkdir -p 0_genomes/reference/

# copy manually genomes folder containing fna, faa, gff, pathways files

### 3.1 Reference generation

For each experiment, the reference genomes and annotation were created:

In [ ]:
cat list | sed 's/_r.//g' | uniq | while read exp; do
    # genome reference
    cat list | sed 's/_r.//g' | uniq | sed '1d' | grep -v "${exp}" | while read expp; do
        cat 0_genomes/${expp}/${expp}.fna
    done > 0_genomes/reference/${exp}.fasta
    # contigs ids
    cat list | sed 's/_r.//g' | uniq | sed '1d' | grep -v "${exp}" | while read expp; do
        grep ">" 0_genomes/${expp}/${expp}.fna | cut -d' ' -f1 | sed 's/>//g'
    done > 0_genomes/reference/${exp}_contig_ids.txt
    # gff reference
    cat list | sed 's/_r.//g' | uniq | sed '1d' | grep -v "${exp}" | while read expp; do
        cat 0_genomes/${expp}/${expp}.gff | sed 's/accn|//g'
    done > 0_genomes/reference/${exp}.gff
done

### 3.2 Alignment and counts file generation

The following cell aligns the genetic reads to a reference genome using HISAT2, sorts and indexes the aligned reads using SAMtools, and then counts the aligned reads using HTSeq. The output includes alignment files, sorted BAM files, and count files that provide information about the aligned reads in relation to the genome's features.

In [ ]:
mkdir -p 5_functional_analysis/aln/
mkdir -p 5_functional_analysis/counts/

# hisat2 indexing genomes
cat list | sed 's/_r.//g' | uniq | while read exp; do
    hisat2-build 0_genomes/reference/${exp}.fasta 0_genomes/reference/${exp}
done

cat list | while read exp; do
    expp=$(echo "$exp" | sed 's/_r.//g')
    # align reads
    hisat2 -x 0_genomes/reference/${expp} -1 4_reads_prepared/${exp}_1.fastq -2 4_reads_prepared/${exp}_2.fastq --threads 16 -S 5_functional_analysis/aln/${exp}.sam
    # sort aligned reads
    samtools view -S -b 5_functional_analysis/aln/${exp}.sam -@ 16 > 5_functional_analysis/aln/${exp}.bam 
    samtools sort 5_functional_analysis/aln/${exp}.bam -@ 16 -o 5_functional_analysis/aln/${exp}_srt.bam 
    samtools index 5_functional_analysis/aln/${exp}_srt.bam -@ 16
    # count aligned reads
    htseq-count --order=pos --stranded=yes --type=CDS --idattr=ID --additional-attr=locus_tag --additional-attr=product --additional-attr=gene --counts_output=5_functional_analysis/counts/${exp}_counts.tsv --nprocesses=16 5_functional_analysis/aln/${exp}_srt.bam 0_genomes/reference/${expp}.gff
done

### 3.3 File formatting

The following scripts prepare the results obtained from the previous cells and format them into the required structure for further processing with R.

In [ ]:
# merges tables of counts
cat << EOF > merge_tables.py
#!/usr/bin/env python

import sys
import pandas as pd

file_df1 = sys.argv[1]
file_df2 = sys.argv[2]
file_df3 = sys.argv[3]
out_file = sys.argv[4]

df1 = pd.read_csv(file_df1, sep="\t", names=["ID", "locus_tag", "product", "gene", "counts_r1"])
df2 = pd.read_csv(file_df2, sep="\t", names=["ID", "locus_tag", "product", "gene", "counts_r2"])
df3 = pd.read_csv(file_df3, sep="\t", names=["ID", "locus_tag", "product", "gene", "counts_r3"])

merged_df = df1.merge(df2, on=["ID", "locus_tag", "product", "gene"], how="outer").merge(df3, on=["ID", "locus_tag", "product", "gene"], how="outer")

merged_df.to_csv(out_file, sep="\t", index=False)
EOF

chmod +x merge_tables.py

In [ ]:
# runs the script
mkdir -p 5_functional_analysis/results
cd 5_functional_analysis/

cat ../list | sed 's/_r.//g' | uniq | while read exp; do    
    ../merge_tables.py counts/${exp}_r1_counts.tsv counts/${exp}_r2_counts.tsv counts/${exp}_r3_counts.tsv results/${exp}_counts.tsv
    sed -i "s/counts_r/${exp}_R/g" results/${exp}_counts.tsv
done

cd ..
rm merge_tables.py

In [ ]:
# merges to a final table
cd 5_functional_analysis/

cat << EOF > merge_tables.py
#!/usr/bin/env python

import pandas as pd

all = pd.read_csv("results/all_counts.tsv", sep="\t")
bbif = pd.read_csv("results/bbif_counts.tsv", sep="\t")
bbre = pd.read_csv("results/bbre_counts.tsv", sep="\t")
binf = pd.read_csv("results/binf_counts.tsv", sep="\t")
bthe = pd.read_csv("results/bthe_counts.tsv", sep="\t")
lsym = pd.read_csv("results/lsym_counts.tsv", sep="\t")
ecol = pd.read_csv("results/ecol_counts.tsv", sep="\t")
paci = pd.read_csv("results/paci_counts.tsv", sep="\t")

df_merged = all.merge(bbif, on=["ID", "locus_tag", "product", "gene"], how="outer")
df_merged = df_merged.merge(bbre, on=["ID", "locus_tag", "product", "gene"], how="outer")
df_merged = df_merged.merge(binf, on=["ID", "locus_tag", "product", "gene"], how="outer")
df_merged = df_merged.merge(bthe, on=["ID", "locus_tag", "product", "gene"], how="outer")
df_merged = df_merged.merge(lsym, on=["ID", "locus_tag", "product", "gene"], how="outer")
df_merged = df_merged.merge(ecol, on=["ID", "locus_tag", "product", "gene"], how="outer")
df_merged = df_merged.merge(paci, on=["ID", "locus_tag", "product", "gene"], how="outer")

df_merged.to_csv("counts.tsv", sep="\t", index=False)
EOF

chmod +x merge_tables.py
./merge_tables.py
rm merge_tables.py

cd ..

The final result is a file called `counts.tsv` inside the `5_functional_analysis` folder, which will be further analyzed with R using DESeq2.

It is also of interest to know the number of readings assigned to each microorganism, the following scripts perform this function.

In [ ]:
cat << EOF > genome_map_count.py
#!/usr/bin/env python

import sys
import pysam

def count_paired_reads(bam_file, genome_ids_file):
    genome_ids = read_genome_ids(genome_ids_file)
    counts = {genome_id: 0 for genome_id in genome_ids}
    samfile = pysam.AlignmentFile(bam_file, "rb")

    for genome_id in genome_ids:
        for read in samfile.fetch(genome_id):
            if read.is_paired and not read.is_secondary and not read.is_supplementary:
                counts[genome_id] += 1

    samfile.close()
    return counts

def read_genome_ids(genome_ids_file):
    with open(genome_ids_file, "r") as file:
        genome_ids = [line.strip() for line in file]

    return genome_ids

bam_file_path = sys.argv[1]
genome_ids_file_path = sys.argv[2]

paired_reads_counts = count_paired_reads(bam_file_path, genome_ids_file_path)

for genome_id, count in paired_reads_counts.items():
    print(f"{genome_id}\t{count}")
EOF

chmod +x genome_map_count.py

In [ ]:
mkdir -p 5_functional_analysis/library_composition

# codes for each contig specified in the genome obtained
cat << EOF > 5_functional_analysis/library_composition/exp_code_genomes.csv
BQJY010000..,bbif
AP012324,bbre
NC_011593,binf
NC_004..3,bthe
NZ_GL834...,lsym
NC_000913,ecol
CP053421,paci
EOF

cat list | while read exp; do
    expp=$(echo "$exp" | sed 's/_r.//g')
    ./genome_map_count.py 5_functional_analysis/aln/${exp}_srt.bam 0_genomes/reference/${expp}_contig_ids.txt > 5_functional_analysis/library_composition/${exp}_comp.temp
    cat 5_functional_analysis/library_composition/exp_code_genomes.csv | while read line; do
        pat=$(echo "$line" | cut -d',' -f1)
        spe=$(echo "$line" | cut -d',' -f2)
        count=$(grep "$pat" 5_functional_analysis/library_composition/${exp}_comp.temp | awk '{sum += $2} END {print sum}')
        echo -e "${spe}\t${count}"
    done > 5_functional_analysis/library_composition/${exp}_comp.txt
    rm 5_functional_analysis/library_composition/${exp}_comp.temp
done

rm genome_map_count.py

In [ ]:
cat << EOF > tidy_data.py
#!/usr/bin/env python

import os
import sys

input_directory = sys.argv[1]
output_file = sys.argv[2]

# Initialize a dictionary to store the data
data = {}

# Process each file in the input directory
for filename in os.listdir(input_directory):
    if filename.endswith(".txt"):
        filepath = os.path.join(input_directory, filename)
        with open(filepath, "r") as file:
            experiment_name = filename.split("_")[0]
            replica_number = filename.split("_")[1].split(".")[0]

            for line in file:
                line = line.strip().split("\t")
                species = line[0]
                num = line[1] if len(line) == 2 else "0"

                if experiment_name not in data:
                    data[experiment_name] = []

                data[experiment_name].append([experiment_name, replica_number, species, num])

# Write the data to the output file
with open(output_file, "w") as file:
    file.write("exp\treplicate\tspecies\tnum\n")
    for experiment_name, rows in data.items():
        for row in rows:
            file.write("\t".join(row) + "\n")

EOF

chmod +x tidy_data.py

./tidy_data.py 5_functional_analysis/library_composition/ 5_functional_analysis/library_composition/comp_data.tsv

rm tidy_data.py

The result of the previous scripts is the `comp_data.tsv` file, which is to be plotted with R.